## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Preambule](#prem)
     * [Package Loading](#package)
     * [Functions](#function)
* [GRU](#lstm)
    * [1.FD001](#fd001)
        * [1.1 Data loading](#fd001dataload)
        * [1.2 Model selection](#fd001modelselect)
    * [2.FD002](#fd002)
         * [2.1 Data loading](#fd002dataload)
         * [2.2 Model selection](#fd002modelselect)
    * [3.FD003](#fd003)
         * [3.1 Data loading](#fd003dataload)
         * [3.2 Model selection](#fd003modelselect)
    * [4.FD004](#fd004)
         * [4.1 Data loading](#fd004dataload)
         * [4.2 Model selection](#fd004modelselect)

### Package loading <a class = "anchor" id="package"></a>

In [2]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt

from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
import keras
from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
import warnings

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
# import keras
import math

from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from methods import *
import warnings
from tensorflow.keras import optimizers
warnings.filterwarnings("ignore")
%matplotlib inline

warnings.filterwarnings('ignore')
print("okay")

okay


### Functions <a class = "anchor" id="function"></a>

In [3]:
# Fixer le seed pour la reproductibilité
SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [5]:
def rul_piecewise_fct(X_train, rul):
    
    X_train['RUL'].clip(upper=rul, inplace=True)
    
    return X_train

def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
    
    return X_train_interim, X_test_interim

# Models functions

# 1layers
def model_gru_1layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model
# 2layers
def model_gru_2layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

# 3layers
def model_gru_3layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

def model_gru_4layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

# GRU

### 1. FD001 <a class = "anchor" id="fd001"></a>

In [6]:
# Data preparation
# Data loading

train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(20631, 27) (13096, 26) (100, 1)


In [7]:
rul_piecewise = 130
train['RUL'].clip(upper=rul_piecewise, inplace=True)

In [82]:
# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(10,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [[32], [64], [128], [256]]

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,4)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh']
batch_size_list = [32, 64, 128, 256]
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options

10752

In [8]:
ITERATIONS = 25
SEED = 0

In [84]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "model_weight/weights_file_gru.h5"
    alpha = 0.3
    sequence_length = 30
    epochs = 25
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_gru_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(17731, 30, 14) (17731, 1) (100, 30, 14)
Epoch 1/25
139/139 [==============================] - 8s 51ms/step - loss: 4484.9697 - val_loss: 2283.7559
Epoch 2/25
139/139 [==============================] - 6s 47ms/step - loss: 2357.8901 - val_loss: 1675.9274
Epoch 3/25
139/139 [==============================] - 7s 50ms/step - loss: 1918.4822 - val_loss: 1566.9929
Epoch 4/25
139/139 [==============================] - 7s 48ms/step - loss: 1413.2267 - val_loss: 854.2256
Epoch 5/25
139/139 [==============================] - 7s 47ms/step - loss: 879.9505 - val_loss: 737.9779
Epoch 6/25
139/139 [==============================] - 7s 48ms/step - loss: 639.4908 - val_loss: 383.0013
Epoch 7/25
139/139 [==============================] - 7s 48ms/step - loss: 417.1738 - val_loss: 237.7495
Epoch 8/25
139/139 [==============================] - 7s 48ms/step - loss: 294.6344 - val_loss: 261.5718
Epoch 9/25
139/139 [==============================] - 7s 48ms/step - loss: 272.3218 - val_loss: 283.1718
Epoch 1

In [90]:
results.to_csv("fd001_result/optim_result_1layer.csv")
results.sort_values(by='S_score').head(5)

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
44,14.261192,0.0,346.845151,0.0,203.381607,0.0,[256],0.3,tanh,32
27,14.123953,0.0,364.835581,0.0,199.486069,0.0,[64],0.3,tanh,32
49,14.355779,0.0,382.946065,0.0,206.088364,0.0,[64],0.2,tanh,32
46,14.615267,0.0,392.689486,0.0,213.606049,0.0,[128],0.1,tanh,32
16,14.602270,0.0,407.648872,0.0,213.226288,0.0,[128],0.1,tanh,32


### 2 layers FD001

In [9]:
#### Model function
def model_gru_2layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
    model = Sequential()
    model.add(GRU(units = 256, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

In [10]:
# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(10,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [[32], [64], [128]]

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,4)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh']
batch_size_list = [32, 64, 128, 256]
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options

8064

In [15]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "model_weight/weights_file_gru.h5"
    alpha = 0.3
    sequence_length = 30
    epochs = 25
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_gru_2layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

In [14]:
results.to_csv('fd001_result/2layergru.csv')
results.sort_values(by="S_score")

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
10,14.114525,0.0,315.553602,0.0,199.219849,0.0,[64],0.1,tanh,128
5,16.640436,0.0,344.560096,0.0,276.904114,0.0,[128],0.1,tanh,256
0,14.643668,0.0,356.079892,0.0,214.437012,0.0,[64],0.2,tanh,128
21,15.545598,0.0,362.195469,0.0,241.665604,0.0,[128],0.2,tanh,256
2,14.640487,0.0,396.175194,0.0,214.343872,0.0,[32],0.3,tanh,64
18,14.175282,0.0,400.861973,0.0,200.938629,0.0,[128],0.1,tanh,32
14,14.060606,0.0,403.599550,0.0,197.700668,0.0,[64],0.1,tanh,32
9,14.648993,0.0,411.117829,0.0,214.593002,0.0,[32],0.3,tanh,64
1,14.421577,0.0,413.279977,0.0,207.981888,0.0,[64],0.2,tanh,64
23,14.306092,0.0,437.686065,0.0,204.664291,0.0,[128],0.1,tanh,32


### 1. FD002 <a class = "anchor" id="fd002"></a>

In [98]:
# Data preparation
# Data loading

train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(53759, 27) (33991, 26) (259, 1)


In [99]:
rul_piecewise = 130
train['RUL'].clip(upper=rul_piecewise, inplace=True)
ITERATIONS = 20
SEED = 0

In [96]:
results.to_csv("fd002_result/optim_result_2layer.csv")

In [101]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "model_weight/weights_file_gru.h5"
    alpha = 0.3
    sequence_length = 30
    epochs = 25
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_gru_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(46219, 30, 14) (46219, 1) (259, 30, 14)
Epoch 1/25
1445/1445 [==============================] - 7s 5ms/step - loss: 4816.7974 - val_loss: 2634.2185
Epoch 2/25
1445/1445 [==============================] - 6s 4ms/step - loss: 2338.8611 - val_loss: 1905.2845
Epoch 3/25
1445/1445 [==============================] - 6s 4ms/step - loss: 1051.1212 - val_loss: 441.3098
Epoch 4/25
1445/1445 [==============================] - 6s 4ms/step - loss: 446.8473 - val_loss: 398.5717
Epoch 5/25
1445/1445 [==============================] - 6s 4ms/step - loss: 353.8566 - val_loss: 356.0776
Epoch 6/25
1445/1445 [==============================] - 7s 5ms/step - loss: 336.4653 - val_loss: 338.7506
Epoch 7/25
1445/1445 [==============================] - 6s 4ms/step - loss: 328.2233 - val_loss: 355.0739
Epoch 8/25
1445/1445 [==============================] - 6s 4ms/step - loss: 322.9799 - val_loss: 376.3207
Epoch 9/25
1445/1445 [==============================] - 6s 4ms/step - loss: 315.9064 - val_loss: 279.1780


In [104]:
results.sort_values(by="S_score").head(5)

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
6,14.061648,0.0,928.483398,0.0,197.729965,0.0,[256],0.1,tanh,32
15,14.831621,0.0,944.086039,0.0,219.976990,0.0,[256],0.1,tanh,32
1,14.895869,0.0,974.758527,0.0,221.886917,0.0,[128],0.3,tanh,32
3,14.676073,0.0,1004.472028,0.0,215.387115,0.0,[128],0.2,tanh,64
16,15.147210,0.0,1048.456973,0.0,229.437958,0.0,[128],0.2,tanh,256


In [103]:
results.to_csv("fd002_result/optim_result_1layer.csv")